In [1]:
import numpy as np
import pandas as pd
import os, shutil, json
import matplotlib.pyplot as plt
import tensorflow as tf
from numpy import argmax
from PIL import Image
from pathlib import Path
import keras
from keras import backend as K
from keras.utils import np_utils
from keras.regularizers import l2
from keras.models import Sequential
from keras.optimizers import RMSprop, Adam, SGD
from keras.preprocessing import image
from keras.engine.topology import Layer
from keras.utils.np_utils import to_categorical
from keras.initializers import truncated_normal
from keras.losses import categorical_crossentropy, binary_crossentropy, mean_squared_error, mean_squared_logarithmic_error
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras_contrib.layers.advanced_activations.swish import Swish
from keras.preprocessing.image import img_to_array, ImageDataGenerator
from keras import layers, models, regularizers, backend, utils
from keras.layers import Activation, BatchNormalization, Conv2D, Conv2DTranspose, Dense, Dropout, Flatten, initializers, InputSpec, Input, LeakyReLU, MaxPooling2D, Reshape
from sklearn.model_selection import train_test_split

abs_path = r'%s' % os.getcwd().replace('\\','/')

C:\Users\Pichau\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Pichau\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Pichau\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Pichau\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: P

In [2]:
Imagens_DataFrame = pd.read_csv("./1_imagens_Mckinsey666.csv") 
Imagens_DataFrame.head()

,path,width,height,size
0,0_2000.jpg,63,63,63 63
1,10000_2004.jpg,62,62,62 62
2,10001_2004.jpg,79,79,79 79
3,10002_2004.jpg,51,51,51 51
4,10003_2004.jpg,77,77,77 77


In [3]:
Imagens_DataFrame.groupby('size').count().sort_values('path').tail()

,path,width,height
size,,,
90 90,1516,1516,1516
95 95,1530,1530,1530
94 94,1534,1534,1534
96 96,1556,1556,1556
92 92,1636,1636,1636


# Modelo 32x32

In [ ]:
class rede_32:
    def __init__(self, batch_size = 20):
        self.latent_dim = 32
        self.height = 32
        self.width = 32
        self.channels = 3
        self.iterations = 10000
        self.batch_size = batch_size
        self.NPZ_file_path = os.path.join(abs_path, "2_imagens_Mckinsey666_"+str(self.height)+"_"+str(self.height)+".npz").replace('\\','/')
        self.create_numpy_file()
        
        images_path = os.path.join(abs_path,'2_Generated_Images_' + str(self.height) + '_' + str(self.height)).replace('\\','/')
        if not os.path.exists(images_path):
            os.mkdir(images_path)
        self.save_dir = images_path
        
    def create_numpy_file(self):
        imagens_x = []
        if not os.path.exists(self.NPZ_file_path):
            Imagens_DataFrame = pd.read_csv("./1_imagens_Mckinsey666.csv")
            for index, row in Imagens_DataFrame.iterrows():
                    image_path = os.path.join(abs_path, '1_Mckinsey666', row['path']).replace('\\','/')
                    image = Image.open(image_path)
                    newsize = (32, 32) 
                    image = image.resize(newsize) 
                    image_numpy = np.asarray(image)
                    imagens_x.append(image_numpy)

            imagens_x = np.asarray(imagens_x, dtype=np.float32)
            np.savez(self.NPZ_file_path, x=imagens_x)
    
    def get_images_as_nparray(self):
        with np.load(self.NPZ_file_path) as data:
            imagens_x = data['x']
        x_train = imagens_x.reshape(
                (imagens_x.shape[0],) + 
                (self.height, self.width, self.channels)).astype('float32') / 255.
        return x_train
        
    def create_generator(self):
        generator_input = keras.Input(shape=(self.latent_dim,))

        #Transforms the input into a 16 × 16 128-channel feature map
        x = Dense(128 * 16 * 16)(generator_input)
        x = LeakyReLU()(x)
        x = Reshape((16, 16, 128))(x)

        x = Conv2D(256, 5, padding='same')(x)
        x = LeakyReLU()(x)

        #Upsamples to 32 × 32
        x = Conv2DTranspose(256, 4, strides=2, padding='same')(x)
        x = LeakyReLU()(x)
        x = Conv2D(256, 5, padding='same')(x)
        x = LeakyReLU()(x)
        x = Conv2D(256, 5, padding='same')(x)
        x = LeakyReLU()(x)

        #Produces a 32 × 32 1-channel feature map (shape of a CIFAR10 image)
        x = Conv2D(self.channels, 7, activation='tanh', padding='same')(x)
        #Instantiates the generator model, which maps the input of shape (latent_dim,) into an image of shape (32, 32, 3)
        generator = keras.models.Model(generator_input, x)
        #generator.summary()
        return generator
    
    def create_discriminator(self):
        discriminator_input = Input(shape=(self.height, self.width, self.channels))
        x = Conv2D(128, 3)(discriminator_input)
        x = LeakyReLU()(x)
        x = Conv2D(128, 4, strides=2)(x)
        x = LeakyReLU()(x)
        x = Conv2D(128, 4, strides=2)(x)
        x = LeakyReLU()(x)
        x = Conv2D(128, 4, strides=2)(x)
        x = LeakyReLU()(x)
        x = Flatten()(x)
        #One dropout layer: an important trick!
        x = Dropout(0.4)(x)
        #Classification layer
        x = Dense(1, activation='sigmoid')(x)
        #Instantiates the discriminator model, which turns a (32, 32, 3) input into a
        #binary classifi-cation decision (fake/real)
        discriminator = keras.models.Model(discriminator_input, x)
        #discriminator.summary()

        discriminator_optimizer = keras.optimizers.RMSprop(
                lr=0.0008,
                clipvalue=1.0, #Uses gradient clipping (by value) in the optimizer
                decay=1e-8) #To stabilize training, uses learning-rate decay
        discriminator.compile(optimizer=discriminator_optimizer, 
                              loss='binary_crossentropy')
        
        return discriminator
    
    def create_adversarial_net(self):
        #Sets discriminator weights to non-trainable (this will only apply to the gan model) 
        discriminator = self.create_discriminator()
        discriminator.trainable = False
        gan_input = keras.Input(shape=(self.latent_dim,))
        generator = self.create_generator()
        gan_output = discriminator(generator(gan_input))
        gan = keras.models.Model(gan_input, gan_output)
        gan_optimizer = keras.optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)
        gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')
        
        return [generator, discriminator, gan]
    
    def build_network(self):
        start = 0
        generator, discriminator, gan = self.create_adversarial_net()
        x_train = self.get_images_as_nparray()
        
        for step in range(self.iterations):
            #Samples random points in the latent space
            random_latent_vectors = np.random.normal(size=(self.batch_size,self.latent_dim))
            #Decodes them to fake images
            generated_images = generator.predict(random_latent_vectors)
            #Combines them with real images
            stop = start + self.batch_size
            real_images = x_train[start: stop]
            combined_images = np.concatenate([generated_images, real_images])
            #Assembles labels, discriminating real from fake images
            labels = np.concatenate([np.ones((self.batch_size, 1)),
                                     np.zeros((self.batch_size, 1))])
            #Adds random noise to the labels—an important trick!
            labels += 0.05 * np.random.random(labels.shape)
            #Trains the discriminator
            d_loss = discriminator.train_on_batch(combined_images, labels)
            #Samples random points in the latent space
            random_latent_vectors = np.random.normal(size=(self.batch_size,
                                                           self.latent_dim))
            #Assembles labels that say “these are all real images” (it’s a lie!)
            misleading_targets = np.zeros((self.batch_size, 1))
            #Trains the generator (via the gan model, where the discriminator weights are frozen)
            a_loss = gan.train_on_batch(random_latent_vectors,
                                        misleading_targets)
            start += self.batch_size
            if start > len(x_train) - self.batch_size:
                start = 0
            #Occasionally saves and plots (every 100 steps)
            if step % 100 == 0:
                #Saves model weights
                gan.save_weights("2_GAN_model_gan_"+str(self.height)+"_"+str(self.height)+".h5")
                #Prints metrics
                print('{} - discriminator loss: {} - adversarial loss: {}'.format(step, d_loss, a_loss))
                #Saves one generated image
                img = image.array_to_img(generated_images[0] * 255., scale=False)
                img.save(os.path.join(self.save_dir,'2_GAN_' + str(step) + '_generated_anime.png'))
                #Saves one real image for comparison
                img = image.array_to_img(real_images[0] * 255., scale=False)
                img.save(os.path.join(self.save_dir,'2_GAN_' + str(step) + '_real_anime.png'))
            if step == self.iterations-1:
                valores_1 = discriminator.evaluate(x_train, np.zeros((int(x_train.shape[0]), 1)))
                valores_0 = discriminator.evaluate(generated_images, np.ones((self.batch_size, 1)))
                print('Imagens reais - test loss, test acc:', valores_1)
                print('Imagens geradas - test loss, test acc:', valores_0)

In [6]:
rede_32 = rede_32(128)
rede_32.build_network()